# REDE DE CITAÇÕES - SCOPUS - LÓGICA

In [1]:
# CARREGANDO BIBLIOTECAS
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import OpenSSL.SSL
import getpass

import pybliometrics
import pybliometrics.scopus as sc

from selenium import webdriver
from bs4 import BeautifulSoup

from pymongo import MongoClient
import pandas as pd
import pickle

from math import *

import os
import time
import itertools
import gc 
from tqdm import tqdm_notebook as tqdm

import configparser as cf

path_atual = os.getcwd()
print(path_atual)
path_download = '../../Downloads/'
path_scopus = '../../../Walter/.scopus'

C:\Users\Walter\Documents\WOS_CitationNetworks


O **pybliometrics** (anteriormente **scopus**) usa um arquivo de configuração `~/.scopus/config.ini` para salvar credenciais e nomes de diretório da pasta que armazena arquivos em cache.

Para alterar sua chave ou alterar diretórios para arquivos em cache, edite esse arquivo manualmente e importe pybliometrics novamente.

Se o arquivo de configuração não existir, o pybliometrics emitirá um aviso. Para gerar o arquivo de configuração, faça o seguinte comando `pybliometrics.utils.create_config()` logo após a importação.


A **pybliometrics** solicita suas credenciais. A maioria dos usuários só precisa fornecer a chave da API e pressionar Enter no segundo prompt. 

In [3]:
# CONFIGURAÇÕES
#pybliometrics.utilis.create_config()

In [4]:
# CATEGORIA PESQUISADA
category = 'LOGIC'

## 1. No WoS, escolher uma área de pubicação e obter a listagem de periódicos.

Usando o Web of Science, obter uma listagem de todos os periódicos publicados na categoria escolhida (**MATHEMATICS**). A fim de filtrar os resultados, foi escolhida uma subárea: **LOGIC**.

* Em `Advanced Search` do WoS, pesquisar: `WC=(MATHEMATICS)` e selecionar a opção de apenas artigos. São encontrados 1.268.522 artigos. 


* Em seguida, selecionar o filtro de categoria do WoS, `LOGIC`. Agora, são eoncontrados 13.030 artigos. 



* No campo de filtros, tem a opção `Source Titles`. Ir em `more options / values...`. Com a página atualizada, ir em ` Analyze results`.


* Novamente, escolher a opção `Source Titles`. 


* No final da página, selecionar a opção `All data rows (up to 100,000)` e realizar o download. 

![](figuras/journalsVis_LOGIC.jpg)

In [5]:
file_journals = pd.read_csv('journals/SourceTitles_' + category + '.txt', sep='\t',index_col=False)
file_journals.index += 1 
file_journals = file_journals[:-1]

In [6]:
file_journals

,Source Titles,records,% of 13030
1,JOURNAL OF SYMBOLIC LOGIC,3944.0,30.269
2,ANNALS OF PURE AND APPLIED LOGIC,2232.0,17.130
3,MATHEMATICAL LOGIC QUARTERLY,1322.0,10.146
4,ARCHIVE FOR MATHEMATICAL LOGIC,1126.0,8.642
5,JOURNAL OF ALGORITHMS,852.0,6.539
6,LOGIC JOURNAL OF THE IGPL,749.0,5.748
7,STUDIA LOGICA,489.0,3.753
8,ALGEBRA AND LOGIC,479.0,3.676
9,REVIEW OF SYMBOLIC LOGIC,359.0,2.755
10,BULLETIN OF SYMBOLIC LOGIC,350.0,2.686


In [7]:
SourceTitles = list(file_journals['Source Titles'])

In [8]:
file_years = pd.read_csv('journals/PublicationYears_' + category + '.txt', sep='\t',index_col=False)
file_years.index += 1 
file_years = file_years[:-2]

In [9]:
file_years.head()

,Publication Years,records,% of 13030
1,2019,141.0,1.082
2,2018,453.0,3.477
3,2017,531.0,4.075
4,2016,523.0,4.014
5,2015,509.0,3.906


In [10]:
file_years.tail()

,Publication Years,records,% of 13030
50,1970,39.0,0.299
51,1969,53.0,0.407
52,1968,45.0,0.345
53,1967,37.0,0.284
54,1966,42.0,0.322


In [11]:
PublicationYears = list(file_years['Publication Years'])

## 2. Usando a API do Scopus, pesquisar os artigos publicados por cada periódico.

Usando a função [**ScopusSearch**](https://scopus.readthedocs.io/en/stable/examples/ScopusSearch.html), buscamos pelo nome de cada periódico (`SourceTitles`) em um intervalo de tempo pré-definido de 1960 a 2018. 

* O nome do periódico deve ser colocado entre aspas duplas para ser pesquisado como uma string, caso contrário será pesquisado como uma interseção das palavras que estão contidas no nome. 

` documents = sc.ScopusSearch('SRCTITLE ("' + str(journal) + '")  AND  PUBDATETXT (' + str(year) + ') AND DOCTYPE(ar) AND SRCTYPE (j)', refresh=True, apiKey=keys[k])`

`documents = sc.ScopusSearch('SRCTITLE ( "JOURNAL OF SYMBOLIC LOGIC" )  AND  PUBDATETXT (2018) AND DOCTYPE(ar) AND SRCTYPE (j)', refresh=True)`

Como podemo gerar apenas 10 chaves de uma só vez, todas foram armazenadas em uma lista para ser usada durante o processo de extração. Caso o valor seja excedido, a extração irá parar e novas chaves derão ser geradas [manualmente](https://dev.elsevier.com/apikey/manage) e substituídas na lista `keys`. 

In [12]:
def createKeys(password,email,op=1):
    # op=1: gera as 10 primeiras chaves (ainda não tenm nenhuma chave gerada)
    # op=2: gera 10 novas chaves (deleta as 10 já existentes e depois faz a geração das chaves)
    
    driver = webdriver.Chrome(executable_path = r'/Users/Walter/Documents/chromedriver')

    scopus_api_link = 'https://dev.elsevier.com/apikey/manage'
    driver.get(scopus_api_link)

    driver.find_element_by_xpath('//*[@id="bdd-email"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="bdd-elsPrimaryBtn"]').click()
    driver.find_element_by_xpath('//*[@id="bdd-password"]').send_keys(password)
    driver.find_element_by_xpath('//*[@id="bdd-elsPrimaryBtn"]').click()
    
    if op==2:
        # deletar todas as dez chaves existentes
        for key in range(1,11):
            time.sleep(5)
            driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[4]/a').click()
            driver.find_element_by_xpath('//*[@id="delregister"]').click()
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="deleteModal"]/div/div/div[3]/button[1]').click()
    
    if op==1 or op==2:
        # criar todas as dez chaves permitidas
        for key in range(1,11):
            time.sleep(5)
            driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[2]/h2/span/a').click()
            driver.find_element_by_xpath('//*[@id="projectName"]').send_keys('scopus'+str(key))
            driver.find_element_by_xpath('//*[@id="websiteURL"]').send_keys('https://github.com/anacwagner/')
            driver.find_element_by_xpath('//*[@id="apicheckbox"]/div/label').click()  
            driver.find_element_by_xpath('//*[@id="tdmcheckbox"]/div/label').click()
            driver.find_element_by_xpath('//*[@id="register"]').click()   
        
    
    soup = BeautifulSoup(driver.page_source,'lxml')
    findAll = soup.findAll('a')
    
    keys = []
    for i in range(10,20):
        keys.append(findAll[i].text)
        
    driver.quit()
    
    return keys

In [13]:
password = getpass.getpass('Password:')
email = 'acwgdb@gmail.com'

Password:········


In [17]:
keys = createKeys(password,email)
keys

['f5864a403461fc0434e84b1d7c1d2e48',
 '838bf38054e9f78bbf383890a5b637af',
 '093c245e896498ae73920aedf1270ed5',
 '14bb24aaae728c626a0e11adf2c0e34d',
 'a8f55b07e4df954138c8055c55541837',
 '69845d3dfc7a3b9be3af6343be4728f9',
 'fde8c4f78a58cb7b698c3b8fa73fe77a',
 '0a3d9afb6dd5a69cee2b593778dff84f',
 'faba95e0f0006c634229394a4aa05fe6',
 '3e3fe942f06880df31d76f64eb9eac4f']

Sempre que gerarmos novas chaves, devemos modificá-la no arquivo de configurações.

In [15]:
def changeKeys(k,keys):
    config = cf.ConfigParser()
    config.read(path_scopus + '/config.ini')
    config.set("Authentication", "apikey", keys[k])
    with open(path_scopus + '/config.ini', "w+") as configfile:
        config.write(configfile)
    print('Nova Chave: ' + config.get("Authentication", "apikey"))

In [16]:
changeKeys(0,keys)

Segue um exemplo da pesquisa que vamos realizar.

In [18]:
documents = sc.ScopusSearch('SRCTITLE ("' + str(SourceTitles[0]) + '")  AND  PUBDATETXT (' + str(2018) + ') AND DOCTYPE(ar) AND SRCTYPE ( j )', refresh=True)
print('Total de resultados encontrados: ' + str(documents.get_results_size()) + ' artigos.')
df_documents = pd.DataFrame(pd.DataFrame(documents.results))
df_documents.index += 1 
df_documents.head(2)

Total de resultados encontrados: 87 artigos.


,eid,doi,pii,pubmed_id,title,subtype,creator,afid,affilname,affiliation_city,...,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
1,2-s2.0-85061941438,10.1017/jsl.2017.89,S0022481217000895,None,Profiniteness in finitely generated varieties ...,ar,Nurakunov A.,60072558;60003675,Kyrgyz National Academy of Sciences;Politechni...,Bishkek;Warsaw,...,4,None,1566-1578,© The Association for Symbolic Logic 2018. Pro...,profinite algebras | standard varieties | unde...,1,0,None,undefined,None
2,2-s2.0-85061938998,10.1017/jsl.2018.35,S002248121800035X,None,Berkeley cardinals and the structure of l(v&am...,ar,Cutolo R.,60017293,Università degli Studi di Napoli Federico II,Naples,...,4,None,1457-1476,© The Association for Symbolic Logic 2018. We...,Berkeley cardinals | choiceless large cardinal...,0,0,None,undefined,None


O resultado da pesquisa sobre `documents` nos permite ter acesso às seguintes informações:

Index(['eid', 'doi', 'pii', 'pubmed_id', 'title', 'subtype', 'creator', 'afid',
       'affilname', 'affiliation_city', 'affiliation_country', 'author_count',
       'author_names', 'author_ids', 'author_afids', 'coverDate',
       'coverDisplayDate', 'publicationName', 'issn', 'source_id', 'eIssn',
       'aggregationType', 'volume', 'issueIdentifier', 'article_number',
       'pageRange', 'description', 'authkeywords', 'citedby_count',
       'openaccess', 'fund_acr', 'fund_no', 'fund_sponsor'],
      dtype='object')

### Mensagens de Erro
Uma exceção é gerada quando o status do download não está ok. Isso evita que informações com falha (ou seja, o status e a mensagem do erro) sejam salvas como arquivo em cache. Essas exceções são da classe base ScopusException.

Algumas delas, são:

 
* `pybliometrics.scopus.exception.Scopus404Error`: **NOT FOUND**
A entidade que você está procurando não existe. Verifique se o seu identificador ainda está apontando para o item que você está procurando.


* `pybliometrics.scopus.exception.Scopus429Error`: **QUOTA EXCEEDED**
O limite semanal de 5000 solicitações da chave de API fornecida (para visualizações padrão) está esgotado. Aguarde uma semana ou altere a chave em `~/.scopus /config.ini.`.


* `pybliometrics.scopus.exception.Scopus500Error`: **INTERNAL SERVER ERROR**
Formalmente, o servidor não responde, por vários motivos. Um motivo comum nas pesquisas é que você usa um nome de campo que não existe. Verifique se sua consulta funciona na Pesquisa avançada.

In [19]:
def allArticlesSource(journalsSourceTitles, PublicationYears):
    
    source_eids = []
    
    i = len(source_eids) + 1
    k = 0 # controle da chave da API
    
    for journal in journalsSourceTitles[i-1:]:
        print('\n ' + str(i) + '. Início da extração dos artigos do periódico \n' + journal)
        
        eids = []
        
        for year in range(int(PublicationYears[-1]),2019):
            
            resultado = False
            
            while resultado == False:
                try: 
                    documents = sc.ScopusSearch('SRCTITLE ("' + str(journal) + '")  AND  PUBDATETXT (' + str(year) + ') AND DOCTYPE(ar) AND  SRCTYPE (j)')
                    eids.append(documents.get_eids())
                    resultado = True
                    
                except sc.exception.Scopus404Error:  #NOT FOUND
                    resultado = True
                except sc.exception.Scopus429Error : #QUOTA EXCEEDED
                    k = k + 1
                    if k == 11:
                        keys = createKeys(password,email)
                        k = 0
                        changeKeys(k,keys)
                    else:
                        changeKeys(k,keys)
                except sc.exception.Scopus500Error:  #INTERNAL SERVER ERROR
                    time.sleep(14)

        source_eids.append(eids)
        i = i + 1

        with open('pickles/' + category + '_eids.pickle', 'wb') as f:
            pickle.dump([source_eids], f)
        
        #os.remove(path_scopus + '/scopus_search/COMPLETE/' + eid)
        gc.collect()

In [20]:
allArticlesSource(SourceTitles, PublicationYears)

In [21]:
with open('pickles/' + category + '_eids.pickle', 'rb') as f:
    source_eids = pickle.load(f)

A variável `source_eids` é uma lista de listas que armazena os códigos de identificação dos artigos publicados por cada periódico em cada ano. Cada ano é uma lista dentro da lista das publicações do periódico correpondente.

In [28]:
articlesSource = list(itertools.chain.from_iterable(source_eids))
articlesSource = list(itertools.chain.from_iterable(articlesSource))
articlesSource = list(set(list(itertools.chain.from_iterable(articlesSource))))
print('Total de artigos da base: ' + str(len(articlesSource)))

Total de artigos da base: 18718


## 3. Usando a API do Scopus, extrair e armazenar as informações de cada artigo.

As informações extraídas são armazenadas em dois diconários, um contendo as informações dos artigos (**info_articles**) e outro contendo as informações dos periódicos (**info_journals**). Essas estruturas foram pensadas para serem utilizadas no **PyMongo**!

In [29]:
def keyNumber():
    # Qual a chave que está sendo utilizada
    config = cf.ConfigParser()
    config.read(path_scopus + '/config.ini')
    #print(config.get("Authentication", "apikey"))
    k = keys.index(str(config.get("Authentication", "apikey")))
    return k

In [30]:
def articlesInfo(articles_eids,keys):
    
    # Qual a chave que está sendo utilizada
    k = keyNumber()
    
    info_journals = {}
    info_articles = {}
    n = len(articles_eids)
    
    for i in tqdm(range(n)):
        eid = articles_eids[i]
        
        id_article = len(info_articles) + 1
        #print('Início da extração das informações do artigo ' + str(id_article))

        resultado = False 
        while resultado == False:
            try: 
                infos = sc.AbstractRetrieval(eid , view='FULL', refresh=True)
                resultado = True
                delete = True
            except sc.exception.Scopus404Error:  #NOT FOUND
                resultado = True
            except sc.exception.Scopus429Error : #QUOTA EXCEEDED
                k = k + 1
                if k == 11:
                    keys = createKeys(password,email)
                    k = 0
                    changeKeys(k,keys)
                else:
                    changeKeys(k,keys)
            except sc.exception.Scopus500Error:  #INTERNAL SERVER ERROR
                time.sleep(14)

        if infos.issn != None:

            aux_issn = infos.issn.split(' ')
            m=0
            n=1
            issn = aux_issn[0]

            if len(aux_issn) == 1: 
                n = 0

            if info_journals.get(aux_issn[m]) == None and info_journals.get(aux_issn[n]) == None:

                id_journal = len(info_journals) + 1
                info_journals[issn] = {"ID_J": id_journal,
                                       "TITLE_J": infos.publicationName}

            elif info_journals.get(aux_issn[0]) != None:
                id_journal = info_journals[issn]['ID_J']

            else:
                issn = aux_issn[1]
                id_journal = info_journals[issn]['ID_J']

        else: 
            issn = infos.publicationName
            if info_journals.get(issn) == None:
                id_journal = len(info_journals) + 1
                info_journals[issn] = {"ID_J": id_journal,
                                       "TITLE_J": infos.publicationName}
            else:
                id_journal = info_journals[issn]['ID_J']

        refs = infos.references
        refs_eids = []
        if refs != None:
            for r in range(len(refs)):
                refs_eids.append(refs[r][1])

        authors = {}
        if infos.authors != None:
            for a in range(len(infos.authors)):
                authors['author_' + str(a)] ={'auid': infos.authors[a][0],
                                              'indexed_name': infos.authors[a][1],
                                              'affiliation': infos.authors[a][-1]}

        try: 
            cited = infos.citedby_count
        except KeyError:
            cited = 'NaN'

        info_articles[eid] = {"ID_A": id_article,
                              "TITLE_A": infos.title,
                              "DOI": infos.doi,
                              "YEAR": infos.coverDate,
                              "AUTHORS": authors,
                              "ISSN": issn,
                              "ID_J": id_journal,
                              "TITLE_J": infos.publicationName,
                              "REFS": refs_eids,
                              "CITED_BY_COUNT": cited}

        gc.collect()
        
        try:
            os.remove(path_scopus + '/abstract_retrieval/FULL/' + eid)
        except FileNotFoundError:
            continue

        with open('pickles/' + category + '_INFOS.pickle', 'wb') as f:
                pickle.dump([info_articles,info_journals], f)
        pass

In [31]:
articlesInfo(articlesSource,keys)

In [42]:
with open('pickles/' + category + '_INFOS.pickle', 'wb') as f:
    pickle.dump([info_articles,info_journals], f)

## 4. Usando a API do Scopus, extrair e armazenar as informações das referências de cada artigo.

In [43]:
with open('pickles/' + category + '_INFOS.pickle', 'rb') as f:
    [info_articles, info_journals] = pickle.load(f)

In [44]:
print('Total de Periódicos: ' + str(len(info_journals)))

Total de Periódicos: 18


In [45]:
print('Total de Artigos publicados na área de Lógica: ' + str(len(info_articles)))

Total de Artigos publicados na área de Lógica: 18718


In [51]:
info_journals['00025232']

{'ID_J': 1, 'TITLE_J': 'Algebra and Logic'}

In [52]:
info_articles['2-s2.0-58049151597']

{'AUTHORS': {'author_0': {'affiliation': ['60069255'],
   'auid': '56494981800',
   'indexed_name': 'Murzina V.F.'}},
 'CITED_BY_COUNT': 0,
 'DOI': '10.1007/s10469-008-9032-y',
 'ID_A': 1,
 'ID_J': 1,
 'ISSN': '00025232',
 'REFS': ['37249034039',
  '27544481093',
  '27544473747',
  '0040575976',
  '84976646107',
  '0005645110'],
 'TITLE_A': 'Temporal logic of linearly ordered α-spaces',
 'TITLE_J': 'Algebra and Logic',
 'YEAR': '2008-11-01'}

In [53]:
info_articles['2-s2.0-58049151597']['REFS']

['37249034039',
 '27544481093',
 '27544473747',
 '0040575976',
 '84976646107',
 '0005645110']

Como podemos perceber, a estrtura da codifcação dos artigos da base está diferente das referências citadas pelo mesmo (sequência apenas numérica). A codificação dos artigos da base possuem a mesma sequência numérica, porém com o seguinte prefixo: **2-s2.0-**. Se não deixarmos padronizadas, como tais sequências são as chaves do dicionário com as informações de todos os artigos, se algum artigo da base aparecer nas referências de outro artigo, ele será inserido novamente no dicionário com outra chave.

In [72]:
def refsInfo(source_eids, info_journals, info_articles):
    
    # Qual a chave que está sendo utilizada
    k = keyNumber()

    art = 0
    n = len(source_eids)
    
    for i in tqdm(range(n)):
        eid = source_eids[i]
        REFS = info_articles[eid]['REFS']
        art = art +  1
        #print('Início da extração das informações das citações do artigo ' + str(art))
        cit = 0
        
        if len(REFS) != 0:
            for ref in REFS:
                cit = cit + 1
                ref = '2-s2.0-' + ref
                if info_articles.get(ref) == None:
                    
                    id_article = len(info_articles) + 1
                    
                    resultado = False 
                    while resultado == False:
                        try: 
                            infos = sc.AbstractRetrieval(ref , view='FULL', refresh=True)
                            resultado = True
                            delete = True
                        except sc.exception.Scopus404Error:  #NOT FOUND
                            resultado = True
                        except sc.exception.Scopus429Error : #QUOTA EXCEEDED
                            k = k + 1
                            if k == 11:
                                keys = createKeys(password,email)
                                k = 0
                                changeKeys(k,keys)
                            else:
                                changeKeys(k,keys)
                        except sc.exception.Scopus500Error:  #INTERNAL SERVER ERROR
                            time.sleep(14)
                    
                    if infos.issn != None:
                        aux_issn = infos.issn.split(' ')
                        m=0
                        n=1
                        issn = aux_issn[0]

                        if len(aux_issn) == 1: 
                            n = 0
                        
                        if info_journals.get(aux_issn[m]) == None and info_journals.get(aux_issn[n]) == None:

                            id_journal = len(info_journals) + 1
                            info_journals[issn] = {"ID_J": id_journal,
                                                   "TITLE_J": infos.publicationName}

                        elif info_journals.get(aux_issn[0]) != None:
                            id_journal = info_journals[issn]['ID_J']

                        else:
                            issn = aux_issn[1]
                            id_journal = info_journals[issn]['ID_J']

                    else: 
                        issn = infos.publicationName
                        if info_journals.get(issn) == None:
                            id_journal = len(info_journals) + 1
                            info_journals[issn] = {"ID_J": id_journal,
                                                   "TITLE_J": infos.publicationName}
                        else:
                            id_journal = info_journals[issn]['ID_J']


                    refs = infos.references
                    refs_eids = []
                    if refs != None:
                        for r in range(len(refs)):
                            refs_eids.append(refs[r][1])

                    authors = {}
                    if infos.authors != None:
                        for a in range(len(infos.authors)):
                            authors['author_' + str(a)] ={'auid': infos.authors[a][0],
                                                          'indexed_name': infos.authors[a][1],
                                                          'affiliation': infos.authors[a][-1]}

                    try: 
                        cited = infos.citedby_count
                    except KeyError:
                        cited = 'NaN'

                    info_articles[ref] = {"ID_A": id_article,
                                          "TITLE_A": infos.title,
                                          "DOI": infos.doi,
                                          "YEAR": infos.coverDate,
                                          "AUTHORS": authors,
                                          "ISSN": issn,
                                          "ID_J": id_journal,
                                          "TITLE_J": infos.publicationName,
                                          "REFS": refs_eids,
                                          "CITED_BY_COUNT": cited}

                    gc.collect()
                    
                    try:
                        os.remove(path_scopus + '/abstract_retrieval/FULL' + ref)
                    except FileNotFoundError:
                        continue

            with open('pickles/' + category + '_REFS.pickle', 'wb') as f:
                pickle.dump([info_articles,info_journals], f)
        pass

    with open('pickles/' + category + '_REFS.pickle', 'wb') as f:
        pickle.dump([info_articles,info_journals], f)

In [73]:
refsInfo(source_eids, info_journals, info_articles)

In [79]:
with open('pickles/' + category + '_REFS.pickle', 'rb') as f:
    [info_articles, info_journals, ref_error] = pickle.load(f)
    #[info_articles, info_journals] = pickle.load(f)

In [80]:
print("Total de Artigos: " + str(len(info_articles)))
print("Total de Periódicos: " + str(len(info_journals)))

Total de Artigos: 187938
Total de Periódicos: 52267
